In [1]:
#Neural Style Transfer based on Neural Algorithm of Artistic Style paper

In [2]:
#We are going to use the feature space from VGG Network and not using the last fully connected layer

In [3]:
#Importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


2023-03-02 17:43:48.919555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 17:43:50.444265: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64::/usr/local/tensorrt/lib/
2023-03-02 17:43:50.444397: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64::/usr/local/tensorrt/lib/
2023-03-02 17:43:50.4444

In [4]:
#downloading the VGG19 model
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')


2023-03-02 17:44:02.132402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 17:44:02.354392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 17:44:02.354840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 17:44:02.355722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

80134624/80134624 [==============================] - 181s 2us/step


In [5]:
#Defining the content and style layers
content_layers = ['block5_conv2']
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']

#Defining the number of layers
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)


In [7]:
#Defining the content and style image
content_image = tf.keras.preprocessing.image.load_img('content.jpg')
style_image = tf.keras.preprocessing.image.load_img('style.jpg')


In [9]:
#Defining the function to load the VGG19 model
def load_vgg_model():
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    #Getting the output of the content and style layers
    content_outputs = [vgg.get_layer(name).output for name in content_layers]
    style_outputs = [vgg.get_layer(name).output for name in style_layers]
    #Getting the model
    model_outputs = content_outputs + style_outputs
    return tf.keras.Model(vgg.input, model_outputs)


In [10]:
#Defining the function to get the content and style features
def get_content_and_style_features(model, content_image, style_image):
    #Getting the content and style features
    content_outputs = model(content_image)
    style_outputs = model(style_image)
    #Getting the content and style features
    content_features = [content_layer[0] for content_layer in content_outputs[:num_content_layers]]
    style_features = [style_layer[0] for style_layer in style_outputs[num_content_layers:]]
    return content_features, style_features

In [11]:
#Defining the function to get the gram matrix
def gram_matrix(input_tensor):
    #Getting the shape of the tensor
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    #Getting the shape of the tensor
    input_shape = tf.shape(input_tensor)
    #Getting the gram matrix
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)


In [12]:
#Defining the function to get the style features
def get_style_features(style_image):
    #Getting the style features
    style_features = gram_matrix(style_image)
    return style_features


In [13]:
#Defining the function to get the content features
def get_content_features(content_image):
    #Getting the content features
    content_features = content_image
    return content_features


In [14]:
#Defining the function to get the loss
def get_loss(model, loss_weights, init_image, gram_style_features, content_features):
    #Getting the style and content features
    style_weight, content_weight = loss_weights
    model_outputs = model(init_image)
    #Getting the style and content features
    style_output_features = model_outputs[num_content_layers:]
    content_output_features = model_outputs[:num_content_layers]
    #Getting the style and content loss
    style_score = 0
    content_score = 0
    #Getting the style and content loss
    weight_per_style_layer = 1.0 / float(num_style_layers)
    for target_style, comb_style in zip(gram_style_features, style_output_features):
        #Getting the style loss
        style_score += weight_per_style_layer * tf.reduce_mean(tf.square(gram_matrix(comb_style[0]) - target_style))
    #Getting the content loss
    weight_per_content_layer = 1.0 / float(num_content_layers)
    for target_content, comb_content in zip(content_features, content_output_features):
        #Getting the content loss
        content_score += weight_per_content_layer*tf.reduce_mean(tf.square(comb_content[0] - target_content))
    #Getting the total loss
    style_score *= style_weight
    content_score *= content_weight
    #Getting the total loss
    loss = style_score + content_score
    return loss, style_score, content_score

In [15]:
#Defining the function to get the gradients
def get_grads(model, loss_weights, init_image, gram_style_features, content_features):
    #Getting the gradients
    with tf.GradientTape() as tape:
        #Getting the loss
        all_loss = get_loss(model, loss_weights, init_image, gram_style_features, content_features)
    #Getting the gradients
    total_loss = all_loss[0]
    return tape.gradient(total_loss, init_image), all_loss


In [16]:
def deprocess_img(processed_img):
    #Getting the image
    x = processed_img.copy()
    #Getting the image
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    #Getting the image
    assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                               "dimension [1, height, width, channel] or [height, width, channel]")
    #Getting the image
    if len(x.shape) != 3:
        raise ValueError("Invalid input to deprocessing image")
    #Getting the image
    if x.shape[2] == 1:
        x = np.tile(x, (1, 1, 3))
    #Getting the image
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    #Getting the image
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [17]:
#Defining the function to run the style transfer
def run_style_transfer(content_image, style_image, num_iterations=1000, content_weight=1e3, style_weight=1e-2):
    #Loading the VGG19 model
    model = load_vgg_model()
    for layer in model.layers:
        layer.trainable = False
    #Getting the content and style features
    content_features, style_features = get_content_and_style_features(model, content_image, style_image)
    #Getting the gram matrix
    gram_style_features = [get_style_features(style_feature) for style_feature in style_features]
    #Getting the content features
    init_image = content_image
    init_image = tf.Variable(init_image, dtype=tf.float32)
    #Defining the optimizer
    opt = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)
    #Defining the best loss, best image and image
    best_loss, best_img = float('inf'), None
    #Defining the loss weights
    loss_weights = (style_weight, content_weight)
    #Defining the list of losses
    iter_count = 1
    #Defining the list of losses
    losses = []
    #Running the iterations
    for i in range(num_iterations):
        #Getting the gradients
        grads, all_loss = get_grads(model, loss_weights, init_image, gram_style_features, content_features)
        #Getting the loss, style loss and content loss
        loss, style_score, content_score = all_loss
        #Updating the variables
        opt.apply_gradients([(grads, init_image)])
        #Clipping the image
        clipped = tf.clip_by_value(init_image, clip_value_min=0.0, clip_value_max=1.0)
        init_image.assign(clipped)
        #Updating the best loss, best image and image
        if loss < best_loss:
            best_loss = loss
            best_img = deprocess_img(init_image.numpy())
        #Printing the loss
        if i % 100 == 0:
            #Printing the loss
            print ('Iteration: {}'.format(i))
            print ('Total loss: {:.4e}, ' 
                   'style loss: {:.4e}, '
                   'content loss: {:.4e}, '.format(loss, style_score, content_score))
            print()
        #Appending the loss
        losses.append(loss)
    #Returning the best loss, best image and losses
    return best_img, best_loss, losses

In [23]:
def load_img(path_to_img):
    #Defining the image size
    img_size = 512
    #Resizing the content and style image
    content_image = tf.keras.preprocessing.image.load_img('content.jpeg', target_size=(img_size, img_size))
    style_image = tf.keras.preprocessing.image.load_img('style.jpeg', target_size=(img_size, img_size))
    #Converting the image to array
    content_image = tf.keras.preprocessing.image.img_to_array(content_image)
    style_image = tf.keras.preprocessing.image.img_to_array(style_image)
    #Converting the image to float32
    content_image = tf.cast(content_image, tf.float32)
    style_image = tf.cast(style_image, tf.float32)
    #Normalizing the image
    content_image = content_image/255.0
    style_image = style_image/255.0
    #Reshaping the image
    content_image = content_image[tf.newaxis, :]
    style_image = style_image[tf.newaxis, :]

    return content_image, style_image



In [25]:
content_path = 'content.jpeg'
style_path = 'style.jpeg'

#Loading the content image
content_image = load_img(content_path)
#Loading the style image
style_image = load_img(style_path)
#Running the style transfer
best, best_loss, losses = run_style_transfer(content_image, style_image, num_iterations=100)


ValueError: Layer "model_1" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor: shape=(1, 512, 512, 3), dtype=float32, numpy=
array([[[[0.6509804 , 0.7372549 , 0.48235294],
         [0.6509804 , 0.7372549 , 0.48235294],
         [0.6509804 , 0.7372549 , 0.48235294],
         ...,
         [0.63529414, 0.7176471 , 0.45490196],
         [0.63529414, 0.7176471 , 0.45490196],
         [0.63529414, 0.7176471 , 0.45490196]],

        [[0.6509804 , 0.7372549 , 0.48235294],
         [0.6509804 , 0.7372549 , 0.48235294],
         [0.6509804 , 0.7372549 , 0.48235294],
         ...,
         [0.63529414, 0.7176471 , 0.45490196],
         [0.63529414, 0.7176471 , 0.45490196],
         [0.63529414, 0.7176471 , 0.45490196]],

        [[0.6509804 , 0.7372549 , 0.48235294],
         [0.6509804 , 0.7372549 , 0.48235294],
         [0.6509804 , 0.7372549 , 0.48235294],
         ...,
         [0.6392157 , 0.72156864, 0.45882353],
         [0.6392157 , 0.72156864, 0.45882353]